<a href="https://colab.research.google.com/github/DerNiccoo/TheEarlyBird/blob/main/Aufgabe3/Nico/3_Transfer_Learning_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
model_conv = torchvision.models.resnet18(pretrained=True)

for param in model_conv.parameters():
  param.requires_grad = False

num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 8)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
net.to(device)

model_save_name = 'cifar8.pt'
path = F"/content/gdrive/My Drive/{model_save_name}"
net.load_state_dict(torch.load(path))

cuda:0


<All keys matched successfully>

In [ ]:
for param in net.parameters():
  param.requires_grad = False

num_ftrs = net.fc3.in_features
net.fc3 = nn.Linear(num_ftrs, 2)


In [ ]:
print(net.parameters())

<generator object Module.parameters at 0x7f677e1f5fc0>


In [ ]:
class CIFAR2(Dataset):
    """CIFAR2."""

    classes = ("plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", 'ship', 'truck')

    def __init__(self, dataset):
      self.cifar = []
      for i in range(len(dataset)):
        if dataset.targets[i] in [8, 9]:
          self.cifar.append(dataset[i])
        if len(self.cifar) >= 50:
          break

      for i in range(len(self.cifar)):
        lst = list(self.cifar[i])
        t = tuple(lst)
        if lst[1] == 8:
          lst[1] = 0
        else:
          lst[1] = 1
        self.cifar[i] = tuple(lst)

    def __len__(self):
        return len(self.cifar)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        items = self.cifar[idx]
        return items

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

train_set = CIFAR2(dataset=trainset)
test_set = CIFAR2(dataset=testset)

trainloader = torch.utils.data.DataLoader(train_set, batch_size=4,
                                          shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(test_set, batch_size=4,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
def train_test(net, lr):
  net.to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)


  for epoch in range(2):  # loop over the dataset multiple times

      running_loss = 0.0
      for i, data in enumerate(trainloader, 0):
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data[0].to(device), data[1].to(device)
          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = net(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          # print statistics
          running_loss += loss.item()
          if i % 10 == 0:    # print every 1000 mini-batches
              print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 1000))


              running_loss = 0.0

  print('Finished Training')

  total = 0
  correct = 0
  with torch.no_grad():
      for data in testloader:
          images, labels = data[0].to(device), data[1].to(device)
          outputs = net(images)
          _, predicted = torch.max(outputs, 1)
          for p, l in zip(predicted, labels):
            total += 1
            if p == l:
              correct += 1

  print('Accuracy %2d%%' % (100 * correct / total))

In [ ]:
train_test(net, 0.001)

[1,     1] loss: 0.001
[1,    11] loss: 0.010
[2,     1] loss: 0.001
[2,    11] loss: 0.007
Finished Training
Accuracy 60%


In [ ]:
# training ohne Freeze
#
#
net = Net()
net.to(device)

model_save_name = 'cifar8.pt'
path = F"/content/gdrive/My Drive/{model_save_name}"
net.load_state_dict(torch.load(path))

num_ftrs = net.fc3.in_features
net.fc3 = nn.Linear(num_ftrs, 2)

train_test(net, 0.00001)

[1,     1] loss: 0.001
[1,    11] loss: 0.006
[2,     1] loss: 0.001
[2,    11] loss: 0.006
Finished Training
Accuracy 74%


In [ ]:
net = torchvision.models.resnet18(pretrained=True)

for param in net.parameters():
  param.requires_grad = False

num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 2)

train_test(net, 0.001)



net = torchvision.models.resnet18(pretrained=True)

num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 2)

train_test(net, 0.00001)

[1,     1] loss: 0.002
[1,    11] loss: 0.009
[2,     1] loss: 0.002
[2,    11] loss: 0.008
Finished Training
Accuracy 42%
[1,     1] loss: 0.001
[1,    11] loss: 0.008
[2,     1] loss: 0.001
[2,    11] loss: 0.009
Finished Training
Accuracy 62%


In [ ]:
net = torchvision.models.vgg16(pretrained=True)
for param in net.parameters():
  param.requires_grad = False

net.classifier  = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 2),
        )

train_test(net, 0.001)

[1,     1] loss: 0.001
[1,    11] loss: 0.007
[2,     1] loss: 0.001
[2,    11] loss: 0.004
Finished Training
Accuracy 58%


In [ ]:
transform = transforms.Compose([transforms.Resize(size=(224, 224)), transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

train_set = CIFAR2(dataset=trainset)
test_set = CIFAR2(dataset=testset)

trainloader = torch.utils.data.DataLoader(train_set, batch_size=4,
                                          shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(test_set, batch_size=4,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
net = torchvision.models.resnet18(pretrained=True)

for param in net.parameters():
  param.requires_grad = False

num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 2)

train_test(net, 0.001)



net = torchvision.models.resnet18(pretrained=True)

num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 2)

train_test(net, 0.00001)

[1,     1] loss: 0.001
[1,    11] loss: 0.007
[2,     1] loss: 0.000
[2,    11] loss: 0.007
Finished Training
Accuracy 42%
[1,     1] loss: 0.001
[1,    11] loss: 0.007
[2,     1] loss: 0.001
[2,    11] loss: 0.007
Finished Training
Accuracy 52%
